# Auto Policy Risk model with sklearn

This notebook uses the sklearn framework to quickly build a linear regression model to calculate auto policy risk. It should run in Watson Studio using a Python 3.10 environment on Cloud Pak for Data 4.6.33. To run the notebook, replace the `CPD_PASSWORD` value with your Cloud Pak for Data admin password in the cell below.

In [1]:
CPD_USERNAME = "admin"
CPD_PASSWORD = "xxxxxxxxxxxxxxx"
EXPERIMENT_NAME = "AutoPolicyRiskFactSheet"

Get information on the current environment.

In [2]:
import os

PROJECT_UID = os.environ['PROJECT_ID']
CPD_URL = os.environ['RUNTIME_ENV_APSX_URL']
CONTAINER_ID = PROJECT_UID
CONTAINER_TYPE = "project"

WML_CREDENTIALS = {
    "url": CPD_URL,
    "password": CPD_PASSWORD,
    "username": CPD_USERNAME,
    "instance_id": "wml_local",
    "version": "4.5"
}

Get an instance of the Watson Machine Learning Client.

In [3]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
print(wml_client.version)

1.0.286


Verify that the AI FactSheets library is installed.

In [4]:
try:
    from ibm_aigov_facts_client import AIGovFactsClient
except:
    !pip install -U ibm-aigov-facts-client
    from ibm_aigov_facts_client import AIGovFactsClient

Connect to the FactSheets client.

In [5]:
from ibm_aigov_facts_client import AIGovFactsClient, CloudPakforDataConfig

creds=CloudPakforDataConfig(service_url=CPD_URL, username=CPD_USERNAME, password=CPD_PASSWORD, disable_ssl_verification=True)
facts_client = AIGovFactsClient(cloud_pak_for_data_configs=creds,experiment_name=EXPERIMENT_NAME,container_type=CONTAINER_TYPE,container_id=CONTAINER_ID,set_as_current_experiment=True)

2023/06/15 03:29:27 INFO : Experiment AutoPolicyRiskFactSheet does not exist, creating new experiment
2023/06/15 03:29:27 INFO : Experiment successfully created with ID 1 and name AutoPolicyRiskFactSheet
2023/06/15 03:29:28 INFO : Autolog enabled Successfully


Read in the training data from the project as a pandas dataframe.

In [6]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

import pandas as pd

df_data_1 = pd.read_csv(wslib.mount.get_data_path('policy_risk_training.csv'))
df_data_1.head()


HOUSEHOLD_ID  POLICY_ID LAST_NAME  ZIPCODE  HOTSPOT1  HOTSPOT2  HOTSPOT3  \
0      AC21568  SX7469732   Sodeman    60404        39        39        35   
1      IL05755  VZ4479371    Habern    60651         4         3        12   
2      HD67057  QY3532256      Veri    60607         7         1         9   
3      GF83176  WQ0600458  Goodlett    60077         8        13        23   
4      HI88786  XS5374749  Entrican    60056        14        20        28   

   HOTSPOT4  HOTSPOT5  NB_DRIVERS  ...  PRIM_DRIVER_GENDER        MAKE_MODEL  \
0        33        35           5  ...                   F  Chevrolet Malibu   
1         6         7           3  ...                   M     Toyota Sienna   
2         6         5           1  ...                   M      Nissan Rogue   
3        18        18           3  ...                   M    Subaru Impreza   
4        22        23           3  ...                   M   Nissan Frontier   

   MODEL_YEAR  INITIAL_ODOMETER LOW_MILEAGE_USE NB_CLAIMS  CLAIM_TOTAL  \
0        2015             64140               1         0            0   
1        2012            127512               1         0            0   
2        2010            113112               1         0            0   
3        2016             45576               1         0            0   
4        2012             53010               0         0            0   

   NB_PRIM_CLAIMS  PRIM_CLAIM_TOTAL  RISK  
0               6                 0    19  
1               5                 0    35  
2               1                 0    35  
3               3                 0    29  
4               3                 0    26  

[5 rows x 24 columns]

Drop the unnecessary fields from the training data.

In [7]:
df_data_1.drop(['HOUSEHOLD_ID', 'POLICY_ID', 'LAST_NAME', 'ZIPCODE'], axis=1, inplace=True, errors='ignore')

Import the necessary sklearn libraries.

In [8]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

Split the train and test data.

In [9]:
X = df_data_1.drop(['RISK'], axis=1)
y = df_data_1['RISK']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15) # 85% training and 15% test

Identify the categorical features, and set up a one hot encoder transform as part of the model pipeline. Fit the training data.

In [10]:
categorical_features = ['PRIM_DRIVER_GENDER', 'MAKE_MODEL']
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features)
    ]
)

pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LinearRegression())]
)

model = pipeline.fit(X_train, y_train)

2023/06/15 03:29:29 INFO : logging results to factsheet for run_id c4144cc67ca84581a4af364db5614433
2023/06/15 03:29:30 INFO : Successfully logged results to Factsheet service for run_id c4144cc67ca84581a4af364db5614433 under asset_id: ee987194-6c95-4d66-bf33-4ccbdad236e8 and project_id : a3a7620a-927c-46f8-9b25-5236d484aa71


Use the model to predict the test data.

In [11]:
y_pred = model.predict(X_test)

In [12]:
y_pred

array([29.3800528 , 30.48450002, 26.81266717, ..., 30.19794894,
       26.81266717, 26.69660445])

Calculate the Root Mean Squared Error of the predictions.

In [14]:
from math import sqrt
meanSquaredError = mean_squared_error(y_test, y_pred)
rootMeanSquaredError = sqrt(meanSquaredError)
print("RMSE:", rootMeanSquaredError)

RMSE: 6.574014612416035


Set the default project to the current project and prepare to store the model.

In [15]:
wml_client.set.default_project(PROJECT_UID)

'SUCCESS'

Define the model runtime and other metadata, and export the information to AI FactSheets.

In [19]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.2-py3.10")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
    wml_client._models.ConfigurationMetaNames.NAME:"{}".format("auto policy risk - sklearn"),
    wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.1",
    wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "RISK"
}

facts_client.export_facts.prepare_model_meta(wml_client=wml_client,meta_props=model_props)

Software Specification ID: b56101f1-309d-549b-a849-eaa63f77b2fb


{'name': 'auto policy risk - sklearn',
 'type': 'scikit-learn_1.1',
 'software_spec': 'b56101f1-309d-549b-a849-eaa63f77b2fb',
 'label_column': 'RISK',
 'custom': {'experiment_id': '2d8d4ac7b30d489c90e92433edc5e7f6',
  'experiment_name': 'AutoPolicyRiskFactSheet'}}

Store the model in the current project.

In [20]:
print("Storing model ...")
published_model_details = wml_client.repository.store_model(
    model=model, 
    meta_props=model_props, 
    training_target=['Risk'],
    training_data=X_train,
    pipeline=pipeline)

model_uid = wml_client.repository.get_model_id(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))

Storing model ...
Done
Model ID: 05006c24-7900-41d7-ab4d-6f14b2a8746b
